In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random
import re
from datetime import datetime

import datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import yaml
from Bio import SeqIO
from datasets import Dataset, DatasetDict, load_from_disk
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, recall_score
from torch.utils.data import DataLoader
from transformers import (
    DataCollatorWithPadding,
    PretrainedConfig,
    PreTrainedModel,
    T5EncoderModel,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    modeling_outputs,
)
from transformers.utils import (
    is_datasets_available,
)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_config = {
    "project_name": "light_attention_localization",
    "seed": 42,
    "weights_and_biases": {
        "enabled": True,
        "project_name": "light_attention_localization",
        "report_to": "wandb",
    },
    "model": {
        "plm": "Rostlab/prot_t5_xl_uniref50",
        "freeze_plm": True,
    },
    "light_attention": {
        "dropout": 0.25,
        "kernel_size": 9,
        "output_dim": 10,
    },
    "lora": {
        "enabled": False,
        "r": 8,
        "lora_alpha": 16,
        "lora_dropout": 0.05,
        "use_rslora": False,
        "use_dora": False,
        "target_modules": [],
        "modules_to_save": [],
    },
    "trainer": {
        "learning_rate": 1e-4,
        "train_batch_size": 64,
        "num_epochs": 1,
        "eval_batch_size": 32,
        "eval_strategy": "steps",
        "eval_steps": 64,
        "eval_on_start": True,
        "eval_sample_size": 32,
        "batch_eval_metrics": True,
        "remove_unused_columns": False,
        "save_strategy": "no",
        "save_steps": 300,
        "save_total_limit": 5,
        "logging_steps": 1,
        "lr_scheduler_type": "cosine",
        "warmup_steps": 100,
    },
}

model_name_identifier = train_config["project_name"] + "-" + datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

label2location = {
    "Cell.membrane": 0,
    "Cytoplasm": 1,
    "Endoplasmic.reticulum": 2,
    "Extracellular": 3,
    "Golgi.apparatus": 4,
    "Lysosome/Vacuole": 5,
    "Mitochondrion": 6,
    "Nucleus": 7,
    "Peroxisome": 8,
    "Plastid": 9,
}

location2label = {
    0: "Cell.membrane",
    1: "Cytoplasm",
    2: "Endoplasmic.reticulum",
    3: "Extracellular",
    4: "Golgi.apparatus",
    5: "Lysosome/Vacuole",
    6: "Mitochondrion",
    7: "Nucleus",
    8: "Peroxisome",
    9: "Plastid",
}

print(model_name_identifier)

In [ ]:
if train_config["weights_and_biases"]["enabled"]:
    import wandb
    run = wandb.init(project=train_config["weights_and_biases"]["project_name"], name=model_name_identifier)


In [ ]:
tokenizer = T5Tokenizer.from_pretrained(
    pretrained_model_name_or_path="Rostlab/prot_t5_xl_uniref50",
    do_lower_case=False,
    use_fast=True,
    legacy=False,
)

if not os.path.exists("../tmp/data/unprocessed/localization/deeploc_data_processed"):
    train_sequences = []
    train_ids = []
    train_labels = []
    with open("../tmp/data/unprocessed/localization/deeploc_data.fasta", "r") as f:
        for record in SeqIO.parse(f, "fasta"):
            train_sequences.append(str(record.seq))
            desc_parts = record.description.split()
            train_ids.append(desc_parts[0])
            train_labels.append(desc_parts[1].split("-")[0])

    test_sequences = []
    test_ids = []
    test_labels = []
    with open("../tmp/data/unprocessed/localization/new_hard_test_set_PIDE20.fasta_rep_seq.fasta", "r") as f:
        for record in SeqIO.parse(f, "fasta"):
            test_sequences.append(str(record.seq))
            desc_parts = record.description.split()
            test_ids.append(desc_parts[0])
            test_labels.append(desc_parts[1].split("-")[0])

    train_dataset = Dataset.from_dict({"sequence": train_sequences, "id": train_ids, "location": train_labels})

    test_dataset = Dataset.from_dict({"sequence": test_sequences, "id": test_ids, "location": test_labels})

    dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

    def process_sequences(sequences):
        processed_sequences = [" ".join(list(re.sub(r"[UZOB]", "X", seq))) for seq in sequences]
        return processed_sequences

    train_processed = process_sequences(dataset["train"]["sequence"])
    test_processed = process_sequences(dataset["test"]["sequence"])

    train_encodings = tokenizer(train_processed, padding=False, truncation=False)

    test_encodings = tokenizer(test_processed, padding=False, truncation=False)

    dataset["train"] = dataset["train"].add_column("processed_sequence", train_processed)
    dataset["train"] = dataset["train"].add_column("input_ids", train_encodings["input_ids"])
    dataset["train"] = dataset["train"].add_column("attention_mask", train_encodings["attention_mask"])

    dataset["test"] = dataset["test"].add_column("processed_sequence", test_processed)
    dataset["test"] = dataset["test"].add_column("input_ids", test_encodings["input_ids"])
    dataset["test"] = dataset["test"].add_column("attention_mask", test_encodings["attention_mask"])

    dataset["train"] = dataset["train"].map(lambda x: {"labels": label2location[x["location"]]})
    dataset["test"] = dataset["test"].map(lambda x: {"labels": label2location[x["location"]]})

    dataset.save_to_disk("../tmp/data/unprocessed/localization/deeploc_data_processed")
else:
    dataset = load_from_disk("../tmp/data/unprocessed/localization/deeploc_data_processed")


print(f"Dataset size in MB: {sum(dataset[split].data.nbytes for split in dataset) / (1024 * 1024):.2f}")
print(dataset)
print(dataset["train"][0])
print(dataset["test"][0])

In [ ]:
train_lengths = [len(seq) for seq in dataset["train"]["sequence"]]
test_lengths = [len(seq) for seq in dataset["test"]["sequence"]]

max_train_length = max(train_lengths)
max_test_length = max(test_lengths)
max_length = max(max_train_length, max_test_length)

print(f"Longest sequence in train set: {max_train_length}")
print(f"Longest sequence in test set: {max_test_length}")
print(f"Longest sequence overall: {max_length}")

longest_idx = train_lengths.index(max_train_length) if max_train_length == max_length else test_lengths.index(max_test_length)
longest_split = "train" if max_train_length == max_length else "test"
print(f"\nExample of longest sequence (from {longest_split} set):")
print(dataset[longest_split][longest_idx]["sequence"])

plt.figure(figsize=(12, 6))

plt.hist(train_lengths, bins=50, alpha=0.5, label="Train", density=True)
plt.hist(test_lengths, bins=50, alpha=0.5, label="Test", density=True)

plt.xlabel("Sequence Length")
plt.ylabel("Density")
plt.title("Distribution of Sequence Lengths")
plt.legend()
plt.grid(True, alpha=0.3)

plt.axvline(
    sum(train_lengths) / len(train_lengths),
    color="blue",
    linestyle="dashed",
    alpha=0.5,
    label=f"Train Mean: {sum(train_lengths)/len(train_lengths):.0f}",
)
plt.axvline(
    sum(test_lengths) / len(test_lengths),
    color="orange",
    linestyle="dashed",
    alpha=0.5,
    label=f"Test Mean: {sum(test_lengths)/len(test_lengths):.0f}",
)

plt.legend()
plt.show()

print(f"\nSequence length statistics:")
print(f"Train set - Mean: {sum(train_lengths)/len(train_lengths):.1f}, Median: {sorted(train_lengths)[len(train_lengths)//2]}")
print(f"Test set  - Mean: {sum(test_lengths)/len(test_lengths):.1f}, Median: {sorted(test_lengths)[len(test_lengths)//2]}")


In [ ]:
max_allowed_length = 1024

train_indices = [i for i, length in enumerate(train_lengths) if length <= max_allowed_length]
dataset["train"] = dataset["train"].select(train_indices)

test_indices = [i for i, length in enumerate(test_lengths) if length <= max_allowed_length]
dataset["test"] = dataset["test"].select(test_indices)

print(f"Filtered dataset statistics:")
print(f"Train set: {len(dataset['train'])} sequences (removed {len(train_lengths) - len(dataset['train'])} sequences)")
print(f"Test set: {len(dataset['test'])} sequences (removed {len(test_lengths) - len(dataset['test'])} sequences)")

In [ ]:
example_sequence = dataset["train"][0]["input_ids"]
decoded_sequence = tokenizer.decode(example_sequence)
print("Original sequence:\t\t", dataset["train"][0]["sequence"])
print("Tokenized and decoded sequence:\t", decoded_sequence.replace(" ", ""))

In [8]:
# print("\nTrain set label distribution:")
# print(pd.Series(dataset["train"]["label"]).value_counts())

# print("\nTest set label distribution:")
# print(pd.Series(dataset["test"]["label"]).value_counts())

# print("\nNumber of distinct labels:")
# print(f"Train: {len(set(dataset['train']['label']))}")
# print(f"Test: {len(set(dataset['test']['label']))}")


# unique_labels = sorted(set(dataset["train"]["location"]).union(set(dataset["test"]["location"])))
# label2id = {label: idx for idx, label in enumerate(unique_labels)}
# id2label = {idx: label for label, idx in label2id.items()}

# print(label2id)
# print(id2label)

In [9]:
def trim_attention_mask(attention_mask, trim_beginning=0, trim_end=0):
    """
    Finds indices of first n and last m 1s in attention mask and sets them to 0.
    Vectorized implementation.
    Args:
        attention_mask: tensor of shape (batch_size, seq_length)
        trim_beginning: number of 1s to trim from beginning
        trim_end: number of 1s to trim from end
    Returns:
        Modified attention mask with first n and last m 1s set to 0
    """
    if trim_beginning == 0 and trim_end == 0:
        return attention_mask

    attention_mask = attention_mask.clone()

    cumsum_forward = torch.cumsum(attention_mask, dim=1)

    cumsum_backward = torch.cumsum(attention_mask.flip(dims=[1]), dim=1).flip(dims=[1])

    if trim_beginning > 0:
        beginning_mask = cumsum_forward > trim_beginning
        attention_mask = attention_mask * beginning_mask

    if trim_end > 0:
        end_mask = cumsum_backward > trim_end
        attention_mask = attention_mask * end_mask

    return attention_mask


# From Hannes
class LightAttention(nn.Module):
    def __init__(self, embeddings_dim=1024, output_dim=11, dropout=0.25, kernel_size=9, conv_dropout: float = 0.25):
        super(LightAttention, self).__init__()

        self.feature_convolution = nn.Conv1d(embeddings_dim, embeddings_dim, kernel_size, stride=1, padding=kernel_size // 2)
        self.attention_convolution = nn.Conv1d(embeddings_dim, embeddings_dim, kernel_size, stride=1, padding=kernel_size // 2)

        self.softmax = nn.Softmax(dim=-1)

        self.dropout = nn.Dropout(conv_dropout)

        self.linear = nn.Sequential(nn.Linear(2 * embeddings_dim, 32), nn.Dropout(dropout), nn.ReLU(), nn.BatchNorm1d(32))

        self.output = nn.Linear(32, output_dim)

    def forward(self, x: torch.Tensor, mask, **kwargs) -> torch.Tensor:
        """
        Args:
            x: [batch_size, embeddings_dim, sequence_length] embedding tensor that should be classified
            mask: [batch_size, sequence_length] mask corresponding to the zero padding used for the shorter sequecnes in the batch. All values corresponding to padding are False and the rest is True.

        Returns:
            classification: [batch_size,output_dim] tensor with logits
        """
        o = self.feature_convolution(x)  # [batch_size, embeddings_dim, sequence_length]
        o = self.dropout(o)  # [batch_gsize, embeddings_dim, sequence_length]
        attention = self.attention_convolution(x)  # [batch_size, embeddings_dim, sequence_length]

        attention = attention.masked_fill(mask[:, None, :] == False, -1e9)

        o1 = torch.sum(o * self.softmax(attention), dim=-1)  # [batchsize, embeddings_dim]
        o2, _ = torch.max(o, dim=-1)  # [batchsize, embeddings_dim]
        o = torch.cat([o1, o2], dim=-1)  # [batchsize, 2*embeddings_dim]
        o = self.linear(o)  # [batchsize, 32]
        return self.output(o)  # [batchsize, output_dim]


class LightAttentionPLM(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        self.light_attention = LightAttention(
            output_dim=config.light_attention["output_dim"],
            dropout=config.light_attention["dropout"],
            kernel_size=config.light_attention["kernel_size"],
        )
        self.light_attention.to(config.device)

        self.plm = T5EncoderModel.from_pretrained(
            pretrained_model_name_or_path=config.plm,
            device_map=config.device,
            torch_dtype="auto",
        )

    def forward(self, input_ids, attention_mask, labels=None, **kwargs):
        x = self.plm(input_ids=input_ids, attention_mask=attention_mask)
        x = x.last_hidden_state[:, :1, :]
        x = x.transpose(1, 2)

        attention_mask = trim_attention_mask(attention_mask, trim_beginning=0, trim_end=1)
        x = self.light_attention(x=x, mask=attention_mask)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(x.view(-1, x.size(-1)), labels.view(-1))

        return modeling_outputs.SequenceClassifierOutput(
            loss=loss,
            logits=x,
            hidden_states=None,
            attentions=None,
        )

    def print_trainable_parameters(self):
        """
        Prints the number of trainable parameters in the model.
        """
        trainable_params = 0
        all_param = 0
        for _, param in self.named_parameters():
            all_param += param.numel()
            if param.requires_grad:
                trainable_params += param.numel()
        print(
            f"trainable params: {trainable_params:,d} || all params: {all_param:,d} "
            f"|| trainable%: {100 * trainable_params / all_param:.2f}%"
        )
        
# class LightAttentionCLIP(PreTrainedModel):
#     def __init__(self, config):
#         super().__init__(config)

#         self.light_attention = LightAttention(
#             output_dim=config.light_attention["output_dim"],
#             dropout=config.light_attention["dropout"],
#             kernel_size=config.light_attention["kernel_size"],
#         )
#         self.light_attention.to(config.device)

#         self.clip = CLIPModel.from_pretrained(
#             pretrained_model_name_or_path=config.plm,
#             device_map=config.device,
#             torch_dtype="auto",
#         )

#     def forward(self, input_ids, attention_mask, labels=None, **kwargs):
#         x = self.plm(input_ids=input_ids, attention_mask=attention_mask)
#         x = x.last_hidden_state[:, :1, :]
#         x = x.transpose(1, 2)

#         attention_mask = trim_attention_mask(attention_mask, trim_beginning=0, trim_end=1)
#         x = self.light_attention(x=x, mask=attention_mask)

#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss()
#             loss = loss_fct(x.view(-1, x.size(-1)), labels.view(-1))

#         return modeling_outputs.SequenceClassifierOutput(
#             loss=loss,
#             logits=x,
#             hidden_states=None,
#             attentions=None,
#         )

#     def print_trainable_parameters(self):
#         """
#         Prints the number of trainable parameters in the model.
#         """
#         trainable_params = 0
#         all_param = 0
#         for _, param in self.named_parameters():
#             all_param += param.numel()
#             if param.requires_grad:
#                 trainable_params += param.numel()
#         print(
#             f"trainable params: {trainable_params:,d} || all params: {all_param:,d} "
#             f"|| trainable%: {100 * trainable_params / all_param:.2f}%"
#         )



class LinearPLM(PreTrainedModel):
    pass


class LinearCLIP(PreTrainedModel):
    pass


In [ ]:
config = PretrainedConfig(
    plm=train_config["model"]["plm"],
    output_hidden_states=True,
    output_attentions=True,
    return_dict=True,
    device=device,
    freeze_plm=train_config["model"]["freeze_plm"],
    light_attention=train_config["light_attention"],
)

model = LightAttentionPLM(config)
model = model.to(device)

if train_config["model"]["freeze_plm"]:
    for param in model.plm.parameters():
        param.requires_grad = False

if train_config["lora"]["enabled"]:
    from peft import LoraConfig, get_peft_model

    lora_config = LoraConfig(
        inference_mode=False,
        r=train_config["lora"]["r"],
        lora_alpha=train_config["lora"]["lora_alpha"],
        lora_dropout=train_config["lora"]["lora_dropout"],
        target_modules=train_config["lora"]["target_modules"],
        bias="none",
        modules_to_save=train_config["lora"]["modules_to_save"],
        use_rslora=train_config["lora"]["use_rslora"],
        use_dora=train_config["lora"]["use_dora"],
    )

    model = get_peft_model(model, lora_config)
    print("target_modules:", lora_config.target_modules)
    print("modules_to_save:", lora_config.modules_to_save)

model.print_trainable_parameters()

for name, param in model.named_parameters():
    if not param.is_cuda:
        print(f"Warning: Parameter {name} is not on CUDA")

In [ ]:
model.eval()

split = "test"

random_idx = random.randint(0, len(dataset[split]))
with torch.no_grad():
    out = model(
        input_ids=torch.tensor([dataset[split][random_idx]["input_ids"]]).to(device),
        attention_mask=torch.tensor([dataset[split][random_idx]["attention_mask"]]).to(device),
        labels=torch.tensor([label2location[dataset[split][random_idx]["location"]]]).to(device),
    )
    
print(dataset[split][random_idx]["location"])
print(dataset[split][random_idx]["sequence"])
print()
print(out.logits.shape)
print(out.logits.softmax(dim=-1))
print(out.logits.argmax(dim=-1))

In [ ]:
class LightAttentionTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        self.eval_sample_size = kwargs.pop("eval_sample_size", 32)
        super().__init__(*args, **kwargs)
    
    def get_eval_dataloader(self, eval_dataset=None):
        """
        Samples the evaluation dataset and returns a subset of size self.eval_sample_size.
        """
        if eval_dataset is None and self.eval_dataset is None:
            raise ValueError("Trainer: evaluation requires an eval_dataset.")

        # If we have persistent workers, don't do a fork bomb especially as eval datasets
        # don't change during training
        dataloader_key = eval_dataset if isinstance(eval_dataset, str) else "eval"
        if (
            hasattr(self, "_eval_dataloaders")
            and dataloader_key in self._eval_dataloaders
            and self.args.dataloader_persistent_workers
        ):
            return self.accelerator.prepare(self._eval_dataloaders[dataloader_key])

        # Use random subset of eval dataset
        eval_dataset = (
            self.eval_dataset[eval_dataset]
            if isinstance(eval_dataset, str)
            else eval_dataset
            if eval_dataset is not None
            else self.eval_dataset
        ).select(random.sample(range(len(self.eval_dataset)), self.eval_sample_size))
        data_collator = self.data_collator

        if is_datasets_available() and isinstance(eval_dataset, datasets.Dataset):
            eval_dataset = self._remove_unused_columns(eval_dataset, description="evaluation")
        else:
            data_collator = self._get_collator_with_removed_columns(data_collator, description="evaluation")

        dataloader_params = {
            "batch_size": self.args.eval_batch_size,
            "collate_fn": data_collator,
            "num_workers": self.args.dataloader_num_workers,
            "pin_memory": self.args.dataloader_pin_memory,
            "persistent_workers": self.args.dataloader_persistent_workers,
        }

        if not isinstance(eval_dataset, torch.utils.data.IterableDataset):
            dataloader_params["sampler"] = self._get_eval_sampler(eval_dataset)
            dataloader_params["drop_last"] = self.args.dataloader_drop_last
            dataloader_params["prefetch_factor"] = self.args.dataloader_prefetch_factor

        # accelerator.free_memory() will destroy the references, so
        # we need to store the non-prepared version
        eval_dataloader = DataLoader(eval_dataset, **dataloader_params)
        if self.args.dataloader_persistent_workers:
            if hasattr(self, "_eval_dataloaders"):
                self._eval_dataloaders[dataloader_key] = eval_dataloader
            else:
                self._eval_dataloaders = {dataloader_key: eval_dataloader}

        return self.accelerator.prepare(eval_dataloader)


training_args = TrainingArguments(
    output_dir=f"../tmp/models/checkpoints/{train_config['project_name']}",
    run_name=train_config["project_name"] if train_config["weights_and_biases"]["enabled"] else None,
    report_to=train_config["weights_and_biases"]["report_to"] if train_config["weights_and_biases"]["enabled"] else None,
    learning_rate=train_config["trainer"]["learning_rate"],
    per_device_train_batch_size=train_config["trainer"]["train_batch_size"],
    num_train_epochs=train_config["trainer"]["num_epochs"],
    eval_strategy=train_config["trainer"]["eval_strategy"],
    eval_on_start=True,
    eval_steps=train_config["trainer"]["eval_steps"],
    per_device_eval_batch_size=train_config["trainer"]["eval_batch_size"],
    save_strategy=train_config["trainer"]["save_strategy"],
    save_steps=train_config["trainer"]["save_steps"],
    save_total_limit=train_config["trainer"]["save_total_limit"],
    logging_strategy="steps",
    logging_steps=train_config["trainer"]["logging_steps"],
    seed=train_config["seed"],
    lr_scheduler_type=train_config["trainer"]["lr_scheduler_type"],
    warmup_steps=train_config["trainer"]["warmup_steps"],
    label_names=["labels"],
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, pad_to_multiple_of=8)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    print(predictions)
    print(labels)
    
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
        "recall": recall_score(labels, predictions, average="weighted"),
        "mcc": matthews_corrcoef(labels, predictions),
    }


trainer = LightAttentionTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["train"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    eval_sample_size=train_config["trainer"]["eval_sample_size"],
)

In [ ]:
# trainer.train()
trainer.evaluate()

In [ ]:
model_save_path = f"../tmp/models/{model_name_identifier}"
trainer.save_model(model_save_path)

history = pd.DataFrame(trainer.state.log_history)
history.to_csv(f"{model_save_path}/training_log.csv", index=False)

with open(f"{model_save_path}/train_config.yaml", "w") as f:
    train_config["model"]["reload_from_checkpoint_path"] = model_save_path
    yaml.dump(train_config, f, sort_keys=False)


fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

metrics = ['eval_accuracy', 'eval_f1', 'eval_recall', 'eval_mcc']
for metric in metrics:
    ax1.plot(history[history[metric].notna()]['step'], 
             history[history[metric].notna()][metric],
             marker='o', label=metric.replace('eval_', ''))
ax1.set_xlabel('Step')
ax1.set_ylabel('Score')
ax1.set_title('Evaluation Metrics')
ax1.legend()
ax1.grid(True)

train_loss = history[history['loss'].notna()]
eval_loss = history[history['eval_loss'].notna()]
ax2.plot(train_loss['step'], train_loss['loss'], 
         label='Training Loss', marker='o')
ax2.plot(eval_loss['step'], eval_loss['eval_loss'],
         label='Evaluation Loss', marker='o')
ax2.set_xlabel('Step')
ax2.set_ylabel('Loss')
ax2.set_title('Training and Evaluation Loss')
ax2.legend()
ax2.grid(True)

plt.tight_layout()


fig.savefig(f"{model_save_path}/training_history.png")
plt.close(fig)

print("Model, config, and log saved to:", model_save_path)